In [1]:
!pip install langchain langchain_core langchain_community pydantic duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [2]:
pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 5.7 MB/s eta 0:00:00


# Built-in Tool **DuckDuckGo Search**

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

results = search.invoke("top news in Pakistan today")
print(results)

/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Top Hat was created by students for students, with the goal of helping everyone succeed in higher education. Whether you're logging in for the first time or looking to get the most out of our … Experience a seamless connection between Top Hat and your LMS. Enjoy easy navigation, direct links to course materials, and synced grades for better teaching and learning. Top Hat ... Top HatLoading Jul 7, 2025 · 1. Top Hat ACE (AI) Top Hat Ace is an AI-powered assistant that helps you understand concepts, get examples, and quiz yourself for exams and midterms. ACE Study Assistance (AI) … Product Overview Pricing Catalog Support Accessibility Resources Teaching Resources Blog Educator Stories Glossary of Higher Ed Company The Top Hat Story Meet the Team Newsroom …


In [4]:
print(search.name)
print(search.description)
print(search.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


# Built-in Tool **Shell Tool**

In [5]:
from langchain_community.tools import ShellTool

shell = ShellTool()

results = shell.invoke("ls")
print(results)

Executing command:
 ls
sample_data



<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
/usr/local/lib/python3.11/dist-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


# Custom Tools

## Method 1:  `@tool decorator`



In [6]:
from langchain_core.tools import tool

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()


In [7]:
# step1: create function

def multiply(a, b):
  """Multiply two numbers"""
  return a*b


In [8]:
# step2: add type hints

def multiply(a: int, b: int) -> int:
  """Multiply two numbers"""
  return a*b

In [9]:
# step3: add tool decorator

@tool
def multiply(a: int, b: int) -> int:
  """Multiply two numbers"""
  return a*b

In [10]:
result = multiply.invoke({"a": 2, "b": 3})
print(result)

6


In [11]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [12]:
# what model see
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Method2: using `StructuredTool` & Pydantic

In [13]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [14]:
class multiplyinput(BaseModel):
  a: int = Field(required=True, description="first number")
  b: int = Field(required=True, description="second number")

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [15]:
def multiply_func(a: int, b: int) -> int:
  return a*b

In [16]:
multiplyTool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=multiplyinput
)

In [17]:
result = multiplyTool.invoke({"a": 2, "b": 3})

print(result)
print(multiplyTool.name)
print(multiplyTool.description)
print(multiplyTool.args)

6
multiply
Multiply two numbers
{'a': {'description': 'first number', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'required': True, 'title': 'B', 'type': 'integer'}}


## Method 3: using `BaseTool` class


In [18]:
from langchain.tools import BaseTool
from typing import Optional, Type

In [19]:
# arg schema using pydantic

class multiplyinput(BaseModel):
  a: int = Field(required=True, description="first number")
  b: int = Field(required=True, description="second number")


/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [20]:
class MultiplyTool(BaseTool):
  name: str = 'multiply'
  description: str = 'Multiply two numbers'
  args_schema: Type[BaseModel] = multiplyinput

  def _run(self, a:int, b:int) -> int:
    return a*b


In [21]:
multiplytool = MultiplyTool()

In [25]:
result = multiplytool.invoke({'a': 7, 'b': 3})

print(result)
print(multiplytool.name)
print(multiplytool.description)
print(multiplytool.args)

21
multiply
Multiply two numbers
{'a': {'description': 'first number', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'second number', 'required': True, 'title': 'B', 'type': 'integer'}}


# Toolkit

In [26]:
from langchain.tools import tool

In [27]:
from re import A
# custom tools
@tool
def add(a: int, b: int) -> int:
  """Add two numbers"""
  return a+b


@tool
def square(a:int) -> int:
  """Taking sqaure of a number"""
  return a**a

In [28]:
class MathToolkit:
  def get_tools(self):
    return [add, square]

In [31]:
mytoolkit = MathToolkit()
tools = mytoolkit.get_tools()

for tool in tools:
  print(tool.name)
  print(tool.description)
  print(tool.args)
  print('---')


add
Add two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}
---
square
Taking sqaure of a number
{'a': {'title': 'A', 'type': 'integer'}}
---
